In [1]:
### 載入套件
import requests
from bs4 import BeautifulSoup as bs 

In [2]:
### 取得 首頁 source code
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")


In [3]:
### 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5 篇
for ele in soup.select("div.sliderBlk a"):
    url = ele["href"]
    links.append(url)
    
# 抓取 6~10 篇
for ele in soup.select("div.leftCol div.listBlk a"):
    links.append(ele["href"])
    
for url in links:
    print(url)

https://www.nownews.com/news/5429797
https://www.nownews.com/news/5429795
https://www.nownews.com/news/5429676
https://www.nownews.com/news/5429746
https://www.nownews.com/news/5429781
https://www.nownews.com/news/5429739
https://www.nownews.com/news/5429738
https://www.nownews.com/news/5429704
https://www.nownews.com/news/5429703
https://www.nownews.com/news/5429698


In [4]:
### 抓取本文資料 -> 收集成一個 function 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }


def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup



url3 = links[8]
print(url3)
soup3 = get_source_code(url3)
parsing_content_data(soup3)

https://www.nownews.com/news/5429703


{'title': '曾任蘇貞昌發言人\u3000黃韋鈞宣布參選新北市議員',
 'category': 'NOWnews 今日新聞 > 要聞 > 政治快訊',
 'time': '2021-11-02 17:14:52',
 'content': '行政院長蘇貞昌2018年競選新北市長時的發言人、立委蘇巧慧辦公室前新聞部主任、總統蔡英文倫敦政經學院的學弟黃韋鈞今（2）日透過臉書粉絲專頁，正式宣布投入2022年新北市五股、泰山、林口區議員選舉。黃韋鈞說，他將爭取民進黨的提名，以及一個為大家做事的機會。\n\n▲黃韋鈞表示，自己會延續過去10年來始終如一的拚勁與幹勁，為地方服務、為大家發聲。（圖／翻攝自黃韋鈞臉書）黃韋鈞說，2016年起，他在蘇巧慧團隊，從地方服務、文宣規劃或是質詢稿撰寫的基層服務做起；並在2018年蘇貞昌肩負競選新北市長重任時，擔任競選總部的發言人。在新北的5年，他深刻體認到，新北市是個讓人嚮往落地生根、成家立業的沃土，但卻因種種因素，讓過去快速前進的腳步慢了下來。身為一個希望能為地方付出的政治工作者，他希望能讓城市更宜居、更安全，而且成為更值得每個居民驕傲的家鄉。黃韋鈞說，從高中起，他就對公共事務充滿熱情，進入台大後，更積極參與校園自治活動，並親身投入太陽花學運前線工作。學運使他意識到，要對政策或制度做改變，還是要進入體制，挽袖做事，促使他踏入了政治工作的領域。黃韋鈞回憶，蘇貞昌在2018年參選新北市長時，他跟著在新北各區衝衝衝；讓家族老家就在新北的他，感受格外的熟悉與親切，也體認到好的市政治理，造福的是一整座城市與世世代代。黃韋鈞說，帶著這樣的抱負，他在2020年前往英國倫敦政經學院鑽研公共政策與行政，與來自世界各地的同學與老師們交流、切磋，也以最新的理論及實務為基礎，了解一個負責任的政府該如何為所當為，學習一個有擔當的領導者要怎麼帶領發展。他說，歷經海外的生活與文化衝擊，讓我體會台灣是自己的根基、最思念的故鄉，更是最深層的牽掛。黃韋鈞說，他選擇回到台灣，為家鄉貢獻心力，期望能奉獻一己之力，成為五泰林的生力軍，讓地方發展、讓五泰林的美與好，被更多人看見。黃韋鈞說，林口做為一個新市鎮，有捷運、大型醫院與學術重鎮，在新北與國際接軌的規劃上，有巨大的潛力與基礎；五股有撐起雙北產業台灣最重要的工業聚落；歷史悠長、人文薈萃的泰山，有優美的在地文化、值得成為

In [5]:
### 抓取 1 ~ 10 篇文章本文
dataList = []
for url in links:
    content_soup = get_source_code(url)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("Done.")

https://www.nownews.com/news/5429797 is ok.
https://www.nownews.com/news/5429795 is ok.
https://www.nownews.com/news/5429676 is ok.
https://www.nownews.com/news/5429746 is ok.
https://www.nownews.com/news/5429781 is ok.
https://www.nownews.com/news/5429739 is ok.
https://www.nownews.com/news/5429738 is ok.
https://www.nownews.com/news/5429704 is ok.
https://www.nownews.com/news/5429703 is ok.
https://www.nownews.com/news/5429698 is ok.
Done.


In [6]:
### 資料輸出成檔案 (資料落地)
for data in dataList:

    file_name = "sample/{}.txt".format(data["title"])
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        
        # 資料全部讀取完 , 再一次寫出檔案
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

        # 每讀一段資料 , 就寫出檔案一次 (不佳)
        # for key in data:
        #     msg = "{}:{}\n".format(key , data[key])
        #     out_file.write(msg)

In [7]:
### 透過 API 抓 JSON 資料

# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

# 2. 向 API 發 requests , 取得文章列表資料
api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(api,headers=headers)

# response 用文字格式輸出
# print(res.text)

# response 用 json 格式輸出
data = res.json()

links2 = []

for ele in data["data"]["newsList"]:
    print(ele["postTitle"])
    # print(ele["postUrl"])
    
    url = "https://www.nownews.com" + ele["postUrl"]
#     print(url)
    links2.append(url)

    print("-"*80)
    
# ele 停留在最後一筆資料
pid = ele["id"]


# 3. 再透過 API 的資料 , 取得下一次的 requests 參數
for i in range(1,4):
    api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }

    res = requests.get(api,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        url = "https://www.nownews.com" + ele["postUrl"]
        links2.append(url)
        
    pid = ele["id"]
    
print("Done.")

5429528
AirDrop成痴漢性騷擾工具！必學「簡單4步驟」擋陌生訊息
--------------------------------------------------------------------------------
美國前白宮國安顧問：中國在2024大選前可能「對台動武」
--------------------------------------------------------------------------------
「這台飛機碳排放量少！」　強生搭私人飛機挨轟雙重標準
--------------------------------------------------------------------------------
港媒：「學生動源」前召集人鍾翰林擬承認分裂國家罪
--------------------------------------------------------------------------------
蘇貞昌稱重啟核四「台灣去一半」　國民黨：嘴砲發電
--------------------------------------------------------------------------------
第13輪明起預約！BNT、AZ上下午分流　預估供242萬人施打
--------------------------------------------------------------------------------
國民黨重申「萊豬非美豬」：大陸都能拒絕、蔡卻要人民吃
--------------------------------------------------------------------------------
春節返台者檢疫期滿如何採檢PCR？莊人祥曝聯繫窗口
--------------------------------------------------------------------------------
王金平坐車遭逼車　徐國勇證實：隨扈亮槍保護特定人士
--------------------------------------------------------------------------------
屢被勸進選2022台中市長　何

In [8]:
### 透過 API 抓 JSON 資料
# 整理版

# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

links2 = []

# 2. 透過 API 的資料 , 並調整 pid , 給下一次的 requests 參數
for i in range(1,4):
    
    url = api.format(pid)

    res = requests.get(url,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        content_url = "https://www.nownews.com" + ele["postUrl"]
        links2.append(content_url)
        
    # 調整 pid , 作為 for-loop 下一動使用
    pid = ele["id"]
    
print("Done.")

5429528
Done.


In [10]:
for link in links2:
    print(link)

https://www.nownews.com/news/column/5429574
https://www.nownews.com/news/column/5429465
https://www.nownews.com/news/column/5429423
https://www.nownews.com/news/column/5429394
https://www.nownews.com/news/column/5429561
https://www.nownews.com/news/column/5429535
https://www.nownews.com/news/column/5429526
https://www.nownews.com/news/column/5429517
https://www.nownews.com/news/column/5429515
https://www.nownews.com/news/column/5429500
https://www.nownews.com/news/column/5429494
https://www.nownews.com/news/column/5429490
https://www.nownews.com/news/column/5429463
https://www.nownews.com/news/column/5429464
https://www.nownews.com/news/column/5429453
https://www.nownews.com/news/column/5429443
https://www.nownews.com/news/column/5429444
https://www.nownews.com/news/column/5429416
https://www.nownews.com/news/column/5429102
https://www.nownews.com/news/column/5429419
https://www.nownews.com/news/column/5429417
https://www.nownews.com/news/column/5429391
https://www.nownews.com/news/col

In [ ]:
####################################################################################################

In [13]:
### 組合程式

##### 載入套件
import time   # 時間套件
import requests
from bs4 import BeautifulSoup as bs 

##### 定義 functions 

# 抓取本文資料 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }

# 取得本文 source code
def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup


##### 主程式 (main)

# 取得 首頁 source code
base_url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(base_url,headers=headers)
soup = bs(res.text,"lxml")

print("* 取得首頁 source code 完成！")

# 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5 篇
for ele in soup.select("div.sliderBlk a"):
    url = ele["href"]
    links.append(url)
    
# 抓取 6~10 篇
for ele in soup.select("div.leftCol div.listBlk a"):
    links.append(ele["href"])
    
print("* 抓取 1~10 篇文章連結 完成！")


# 透過 API 抓 JSON 資料
pid = soup.select("input#txtPageNo")[0]["value"]
api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}"

for i in range(1,2):
    
    # 靜止 0.3 sec
    time.sleep(0.3)
    
    url = api.format(pid)

    res = requests.get(url,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        content_url = "https://www.nownews.com" + ele["postUrl"]
        links.append(content_url)
        
    # 調整 pid , 作為 for-loop 下一動使用
    pid = ele["id"]
    
print("* 抓取更多 文章連結 完成！")


# 抓取文章本文
dataList = []

for url in links:
    time.sleep(0.5)
    content_soup = get_source_code(url)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("* 抓取文章 完成！")


# 資料輸出成檔案 (資料落地)
for data in dataList:
    
#     file_name = "sample/{}.txt".format(data["title"])  # Old

    file_name = "sample/{}.txt".format(data["title"].replace("/","-"))  # New
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

print("Done.")

* 取得首頁 source code 完成！
* 抓取 1~10 篇文章連結 完成！
* 抓取更多 文章連結 完成！
https://www.nownews.com/news/5429896 is ok.
https://www.nownews.com/news/5429849 is ok.
https://www.nownews.com/news/5429847 is ok.
https://www.nownews.com/news/5429840 is ok.
https://www.nownews.com/news/5429839 is ok.
https://www.nownews.com/news/5429763 is ok.
https://www.nownews.com/news/5429838 is ok.
https://www.nownews.com/news/5429797 is ok.
https://www.nownews.com/news/5429795 is ok.
https://www.nownews.com/news/5429676 is ok.
https://www.nownews.com/news/column/5429660 is ok.
https://www.nownews.com/news/column/5429651 is ok.
https://www.nownews.com/news/column/5429617 is ok.
https://www.nownews.com/news/column/5429609 is ok.
https://www.nownews.com/news/column/5429536 is ok.
https://www.nownews.com/news/column/5429538 is ok.
https://www.nownews.com/news/column/5429528 is ok.
https://www.nownews.com/news/column/5429574 is ok.
https://www.nownews.com/news/column/5429465 is ok.
https://www.nownews.com/news/column/542942

In [ ]:
################################################################################################

In [ ]:
################################################################################################

In [ ]:
################################################################################################

In [15]:
#####  台彩大樂透 (當作重新開始爬蟲)

import requests
from bs4 import BeautifulSoup as bs

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

### 發 POST requests 取得頁面 source code
payload = {
    "__VIEWSTATE": "LwgicUIzJstaCahynD+bHwYivALBXSnGZ9jz0xSJytu935YiYfENQYixXD9ExmyrTOGzyjVNrA3EFrEdWLm+LlHzISpPdoCPpQtwrdIgFMiOFvHsiHBTJs6YlekjwJTHzbmWXz5LZIwXsj6SDGHD4fUraSDXPcTOqcdo15P1VTXQXzf0LAXYIHL3ZC+bZoPkiHSdJj27oowb7kfHHnGcLybdkHusGBvl97U6aNPDa3n29k1LzoguOJfjU2bRtfP+qG2txu9WnNhXiKTG9CIGvpHU1NgTqVEbHCi3th3+52n/USqYRiCc705Ji61C0TOrhoS6GCQ5ZGqkWHwYd+W10Maj9PM3NcDYLQo9bRe8oVv9UpUdhCbcbZ0mRCobIGIf/C2fg+DEh7e+KWWHxnNp+InVjXxZ+z9r4jA27zc9Qo18N+0izfhmUPW2Y9Ox65drG4/77cTzXpPs/epeox363vbpla7171p+0Ibc3p4/ikQvmvHiJAGmUDzHIa480rdFjGBNpbU81JUHNYSCBgaaSMLAtwcvztTzhmLbSBKldUAgIxrkIYbwYILvm5A73zmGxDNsRCyUzCV18Ge/r+RVGDcY6jOrET5LBoBX3jhmHbAtHcFy3x7kdW13IXaS97l43WAYCGGYZPwoje+RF8QF1U/vmeUVM2l5ZQfuqnPfeWTdGdmbnFO4VuVPFcNVeDs/k4gQCUYQtu+OG45KrG9HsbANpetPf7TmJ+i+tyLbbf4Hn/k5qjjgijLzeELgDJvK5C6lQ/QdTKNu0r4WKHuW9dKHveqCD2zk/uBNxrYTMhTdR47B7kVKF9kezWmDS6N85+TS5kqkjnCtmdxYdJNMtGYEANYfGGhav3Mv4jWKobI/n86RZISmaP1LXywRxHEOa5pNYr2D2SNifhml8BxJwEfqec19Rp2W3rUcHHF1Fgu9QvblNJaKrElhvlTNJL30nZRTrgWS5CztYJ0c8rXRA3++Qqd7fqm23j4ToeagdlwAl47RuD4QUQ3dvb/s5XX9vvcNGW9fgAazj4yvfPJkjHyVW1DEn15pRsORQZMg4MtuKHnZJIJvEK7HMfRXg4xVEvWnFMeJJDOYMS7U/l6QnkUWbutzLLebUSVy/2Pi20QB/Fc8jImBh5JnsAZ82L628abc8cWlrkl9M78VaubBx8UVmcVuYfs4sf81DhSZvjmEKK/Bx4eYP9SFxHCwcYjEl46pC+hYuO1EGK5QqcYySzLWj1ZWuIJoHajav/rxpNVuHNwXGQ1Smy6OO4yxBCr36vtLIzb6h4VrR4eEaBWKzT1KVx4CSUHXz2sENrx4FNpku645wiZFPhvbaEbUjpFNVFCs+JBmN50aeFp1SpN2N6d5zmFnPdUuDC0qUR26iDjXSAyXXIIYyucqcKLr7vx7U7WL3gaUXug02hb+MFLApUGBxFY2G0bqgf2rNhJlvnYoyPGAW1+7uHWN1cbzC+uhAgNVQzH95oXJugSJVlgu2EHDIC8Qi0LR0v1u91sI+BRMdbfeKBjzf72YKV15rFtqx5O/EHFWfRxXJThP7tRFf3opOvRzVOfThJMZDfS81GldnFfRjGqzHlPorxISbttXFFrDTzvwpA4nyQkBOrb3/2i9udactOJcIdb6306LczIAWD5FcyiE32NPdHvQfkPiWepIygM0ywHW0ZaYTaWX8STnjCn8HwTJySWoKtj/8BvyVnw+GNQ+DKopdoHWDfuKClpE7WzTXmUDBz7GITul5dDXOgeY45ObkmkisT/y8EHEGn7qtdb2HAsOlMcO6iy4bI8ff8E96dPkqRCPjUrALSZX2K2sZdf8WGegw7BVNNlOtQMdhMyiJVUN2tYAPAcXC0voO2QuPv5V+pRvBbmAs+0o/zWhMug9U6IBg0WA92qtDXVsg5JeQbW+etd6eow2o08o1ptQ6sSwYAW55UJyNIjdrxweZpN+piqTrPFDIoHMM9odA9dtGOkS8ivJYuuRVyL6smMgwpj9whQmIRG4Ocud71rPOJMu42d6/kSqSnUD8EDASzM9Vz4D8xAT5dSlhzu1OlrArgbS1LmTVzFeyqmWQGgLl+/xPT4/LFeHOMS6ndT/iOc0RD7aiV8eXJWjiDZtQ3bNpAjg72YcoIj+fGV67T0nKr94MM+hwSdIxoEym7KmvzAd3g/PbSpv9GsjIShpIwENah79nK27EVC0+a4jkthnjxij4vHjCgit2/DZkk3slOhcEOYmbha7Xc/XJICXFPb2eOR1+u9yLGmPPgvZ799KkYAfdVGaqXgHtv3OwTU1o4IzlMlHmkB0USK+sjJUxVqtdZsS/SOqXGp9kf4RcdGDZ35SS+8f/LJp3VchAOn/ZISVfNWBA2j7NS8Pyvtm760ZuR6GV52Ets0M/CkLN8sqI55Csq/ZBhg0aeYGKK1//L/G6JSx/hRipGJ8upNhNAnr7Wl+FTrY6cMrZeAHrLGsVVR9ACpcNw+iaR28nwrjA/uwrw+0QWPu9uoEbM0Ys13f3fim0/11yEP1WIapwZMYLXq6Hn7COdYRAIR8H58GD2Epce16ieM7ZWfPNP4RrG2x59GXkw2N0Wc4KZrPsZ1xOWodxuplexBj9zt4CdWAF8evoTZ/6XjQetVYupjeCHEAwRqZLUmzgTIarMqPmhrGfub53Eo9Y8pgn4RWefhWYrDMRWrdXuiChJGgSqJJXQgoHhs0iW+2k8JUE295K7SkkY3v7oSGCtn6YPA2gV+HpeBkmDcZpQd5bmQ84N2eu7OieQr2fXHsSpPNzQoPmoL5emV4N2nl8LInODRReB6w9Ue8eWaL22AU8B3Nvxj/Y5LY5edLQT/xfIRlBZSsQp2TD79177JlcX++ECJTDORy8CF+FojXE+wIvWgo7uxvOX9JV+FQVBBHb3IexMz4ordRR19BZ/zIeiYyns8bPzCTiH1cYIlEcdE8QpP+EQK1xO2ritcvBYca+bTEfgaNXnN41efyr0ldXIBRRCst7Imdc45sZZBpOdTnSmQ9vevyvSAGfRsEynnb7iIr7nWECpiw0FfOSZJfKT9CzcswgDCQ+2XWY/PksDDThs8WrFRyMEHeeRo1hIXwOy+Lwcyx8SSN2+t5ioJLvTqT2dut3DCeV43M44p2U3KiVp+VSazUKkBeRefw5JXbP98liHElugfok0h3+JbJkmCObqx3g0g0T/v42V4NmAGIQvDWIjlclX5alapoS3e/Bq+LcBz1LdptuWNbphoHURW7oHS9BBSWYzFlwOv3/pMlZhC1Tao2ksvMZfkebdl0+J7RIhdBraEzMkpjo4vFJggGLcR2nsottaTxaKHD81MR7MiCbxIlLp6v3O2bRJbrBcdLJT0sWzKuqMQPyAZO8HGcVsn6/4QyxiWKTejBNUvvKCJzpxTU70ySeMk3VhWg2dsZ1uiptDtgOKmWwjUo7IJr0ZQZv88grjhHruEsfJcU1f3wnaQWdzVO9T54kUYnG+E6JdgHND1EZy1B9A5vMVNumWZNx46MHG/xImq+ceUcoIWGQsbB0P6kD5cMjpuGKhcE8Z0ikpq+UM2/6wzWSNWdOdcR4PNOzzBCgh4qTNWxNXWHKGc1hzftTHvGF1E4+Ta7flqy4aJYh2S9KEufETw2f/azQZB6yWX9mO72TiqHHlYvZWa7FHJuphZ1t2l7pF1GuIqYxOzf35yoDvuFXHDYJFNFAO3sVAn6LFtBbsbql2m+/PSd2XguhdbA+cxXoWGu+YbHyOh1ml/AFkFtN7F0io8WtiKpfbBMPn/n5W8KaXx4Bl+ewCG8m8XDXipWGqVzd523ngXYaUak/lkXZJoRnopJFy/04AhTEy0g/hYcgp86aGIKo9s4lHdGa3v9Z+4Pd7BDcx8WOK/dNyv1uSe0A1Rnt+bt08BcTDeyCJNn1pTrs5koEhgDIlYglc6DYXQPyu8C3BmG4zhF7POZ2XpMlDDtM6C19wUSsGFQRXhH49OBWqBMbEzw5m7otwdzXxD4D6+ydu9s7JTWfeXTw9xtsAvejo6sBJq5gqWlqNk/Oe8SfbIitp8jjx0CB/Tl1l8VVRdkpWhUHJkfZ2JztgvGK0QP8Nyb5XfC0c7uX3PNOPzZ8/liUbAZ54SMwNozo6l1pKh9h2PRsLlamZCTGoJOrvWwTM6ya281tg7qPHQHxwPa1RovhhI+26AYTvSd4I5jPq3P2c6ilQoF25IPydU/GmqHbzioJejNmNb8Fh4t71KtJamkK6IKhutpLcDzs8PHQq12TMTuD5n3GAePPQKxlnfGzXZwY430u185MZukDuhwBHtrw7zsS/s+BClKQ7pY2m2SmAQurZnXMyj/8yQJ7n0ISXqwPYrlNnjGDBlz6U+FuPySyj0rtLBJQsnh2/P/y5ueD5LP1UPHQBx5vUhw1AEHKYMW5f4muiGJ6Cp1ep/LjBWf4PZ+p4tAVbw4c1h7J2+AImYM3tQBEd6X+B8cl1tofAMUxezZLctL9fhGn9k009mBb/Ua5yD6nvSPzqZg3ipcaTiotmed82RuazUfsoBY01G4PvwJdI8qIG1R+y7fwp8X5LooHCdPTV1BBVN5zQ8HISKbrcAIyJx6W168HkVpciR0poV/zTgTftRhRwVwFm1e5RheOvSm1fbEuopPBfOYuMrOh3wKK4hyeocAwwUFV3/490dWi100LX9o6eMFgLwZKS111k2LuXY9gw5sU/p35OCTu1JfHBfPCjV8CXPgYe6VTS/+dp5kvQXCgxVeKL0zY/lKGUNOr8bpf0HNrA8nkkdUrHJJ7vCss5DVbsBT/NZkGqs9pNIY/RSn6rrowW0tNzmcXGl3t1oUWR1SHJ3VM4/Pvhtq4ygAy2z7d2diBvWiQcXbgXDLs2tUrCljt8Rw3D52qrTM4ayxETgR4f3iQEdUaOE9ZgzcRxpLZTxoj0r7AFs2YahHszm2UNzcQ/xyJGgymoteNJmfWm1vNuWIGUzMxmZkeLj55jMebx9N2DO2YcXbSoWSMv39KU9jQXwO86kJo0N+5PNoNOirlhhEQANyH6MeJ9Pzc7PmDW3PDM3GkGC5NnSA8Ul2Pt2TpIn4pC4jb/aj18zMwbZGhaA5zD8mdzFeTgRk1HeZj95yEoMIakHnIeTtoyG6iUbhQsPYp/CDVD9ft+t1x1V2fxgWuxCwFVbWJJkHklcPIcBb8faCz27xRj6cPWBFABYT2cO2ZfEt0AvVlirCfHB77lNnbBOQH91BNPzMFFyujsIe9rOnBHBmBKiObTjQwVqGKoVEXK0hV3TnDbsvwGVmPlXSuvtXts6lVy1wV1h90DtPNZUXOUt4P3577ZRcPOkXnYEmfklNUeoWsFFYzeJE4FyjFP6Om4+EpW61jWjVbxzfheQJYc+DmgjZtuRwSvzAr2YGirWEe9D3xzYgYWgr05y7qmm8o990HKR6OZFVWo9JVqq4yOMwa07/kF/ctFZCoyKWtHzExVhGnkYeFh8BDxqXwg/TM0/exoxVgYhwDO8+eEr6SrpDIaWXncVKhblUKgE8Ufic7+oYnzdCcE5A6R5O4UB3oXwNT+ClDMXl8/6SI4H4BRwqM7NMI0gJ9eZpKl5Gk8EKUk49QmIjh/xfS3854QyV6VO1Cdmvnze5zPqxtNj8xeJB0r3gS/qDYQbx+TMgR7n2u5BJVq0vCWO/fTqz//P/jKrWov2GV+AIm/vzAvVU9JzxLya/oyXOBb3g3dyTKMnTVVloSAlH1XF+J3NfUJI4UK2DmnaCRknw9Q8z9k7WdtmyqgQecnNOk/536H9r4V6udhMfwLAksLIBuLrK85q+GWMiscrn98VSO0+bVeI2x0CZ0zjyBkZejB17iZ+HTHgaDnmzGXvbWvQYhbBFdwxRzlABZqkPbSMEYjT3fF8LHPnn5fVzyuF/H/v+qCArgAIgtsX0pjpaIE3b+lcSCXdcnRkIRmKT7oDgcUBb0TmBl70QfcSccUFhDEDBggqo4gjARAN31ZOGEnewqwsTn7BlKsmIr5Gz9iDfFt7d1FvA3i4v++30OtY7BdApSmoPSpwnrrEVHN24VslKaO78lMgWXbjKWpRD86BYzSyZREq342xy6auMI27I8wUnnHODLpwqLiVgyO+/yLEUeDJ7DZY1FuKkB7iind6J6MJDFdlQpDibIR2G28ZwOQezo0js/N9A2DonX56I0INYYW6ChSEmO4IHy3ngQc0pUkvedNxEHS4th007ZoyA1rekiH3CsgrXHkJzD8FuifaCTZVmy1HseOsvHwkoDOfaVBc8e0wYAgOllzTf17bJXTIgkWnEnFIPfZzAiLSU8Z0cavVjmSfA99DtDfSZ95tvK/0oiX9FfM5sdKof+bUZ3lgkkl76hURjelDaXyk8OKKvqUOpkQN93Je2zg69mEGG0aHUtTrhwPo0fTTx7i/K+6+1wJN27N0Om/aeTrCj9nBilwsjiL18E9AkzMOiC8mR7cjChp3amYpToUGc6apmbUAVkR7ayubjDSfeEwDK9alR5b22uMQIxq9ZliLWLIyVM9tdYrKvJz2QYxYFHhMLS4Epk2f1CnNEAh7xst3dHWxr62OE9i5BxZsJAPgJiT8QfrXVAlAcS33CBGJwoznyrvrGODXVxaCZOfiqRymEhCzB9G0ZA0bssU+39efX/+a7Xnk+EqfNXvnOesUtdeNCBm85yfPdjl3Vfi05Nik0Y/sjmqGM21000JyAzZd7FVhKVqLHnpTQ2uhKw9dnqtCciTdVxzjbjvegjvvLH0eCmhN2uaSFetwYow9i7J+21zJLS2wAmxkFKtvz8PNS5vIVRzDYS3z0JdUmNdvxrclDf4rHTclsMyjXjXGiZiMI54uFtw/XGuGvgsUpCF/TBcnPACMSScx/KbJbmv2BRx2tpBrrYr5+TXPIMUFzzPmZgxshG+wsyPjQVNXTPDph+0vjEbx2cuBgXqJ5FAH1TOriBxxho/62X3OF/1gPRidHTTzwjh0syteIaWYMpSn8qKUnNJXGfx8Fqj4rkM/vdSlcZGACaWalUfHqScTlV7p/O34NLYa0b1ewu0qc347teqe3hMQiL6tNi+67OWyNsh7QCUKN1Sfotu9fwTlOKWYkFAJJs06L6zN6EBqJAj64/AP0TN5p2hye4yQEQgKg5PPwDGQCzpOqHKmmW20JMwV9M6yWdyv6TFDk1TcwLK6yWMg6GF8nCx9XiYHfku26D/kAGmJ564CP5Y3vBWzgQRwAmTwaUcibF32W+xkYPPBp1vT+tZdH8LT8ZqwIZZd4K6cIZD/AX4dVxBhPkp46e8B+M5/qlGEUI/LBHWrlDNu6F8E3x4noROKgznFhPkvCTIg4Fp8cMyi1KVl+ec+rtfxc3kqdA7X2VDyWOGlH2V3gi4Xm5s6qKPyGYJC7LaGxCpKV/NeInumijsevgGpwzDyhuyKEOfhvKIerHhGlGdlBG6KAFKyg3g0DPrGLwhOvU7VCSKpY9yqyTtUEr4W0pKu1vJgDjMSkmlBxh5I5uymN5bsFuSUydqgU8JkUwmRx8lSd/6qkC+6qm+MMPR/j09iuCLDS3kZChU7VQeOAWkYcMlSQECf48AbP3UDd0v7fJZmCNTD+kBOdoMlzTwBTceOe+eZ1xNsovvtXHDSfI+Jpv3XzMoF8zwd4zajC6fq0JdrdkHNXRzBZI21Wd8yPezbS08Y495zpRZJixmDz8Dp5rHlGm17ETMsKNhiBXAOmOR/aI+QR3s+/CU7WYjtX70gaPzp9Nc7679a7WaJ9zQEUaDRsqpvIVGjvn0UJMUViAoytypaz8PVuXBQM//h3skA7p+inmy83VdtpwgBOGwB8Ziw22ljAoJVg7puTET7EdHRylg86q4xZ1E0ENLJWjKoGTHcgfGqK8if65VH5onohLg4Y4iUMO/0UKJtmPYxtDexmvINQ31SOd8sdxVB1PRPciFfRudLfQq4Nqw3RdFpSY2gTgGlt5sGJpFfkC/BMIvMTNXB7Bl93/Dc=",
    "__VIEWSTATEGENERATOR": "11E838D2",
    "__EVENTVALIDATION": "cKlS6TI+ZsyF9uK6yZDpIQXkQdDZ6WlUDBC6uQo0dz9cnfoF4O/K1jlbD9xKU55rjRWBAOfaCP9WDQo0dqKT3UJeNKj4ChyZqMpIaEfUFBlpqjs7f/6X4GjyaGCfhdX1gxFsLfPPREKbxI/0k0gEMQCz4LCpIKJq/drgp0BGIS6wZL/2WCud1xt+RHiEAJvqtNuy3UJO1rcO8o7GRZc/Ic04proPNgN8E+8Eu5Ynz3YH6mVLbOQkfxdbPWudpK4eGxm9Xlc3YCnn9uNC1WB4N9YUqS3TlHgGM48DnUh3t5DOcsCrQOOpNr2+crUHWLJOrAR8nvr5J8uz53qxRv6C00MyIyMteZEpwFh+mguRvz6p1AWzlSKUwUQKfNr0YjSU7mEzGIY/waGS4BQIsvQ1Odfljn99LJsI18EYe8k2WFqZjA70gJFS2hDQQC3Ke8sDoXeOtL/Jmu3kbNFprDvceBQpU7u0txVgBj1v7FtEoAlff5BT//wblVQIqXXLAr+aIMdQC/HYqOReGPNy7eaYw4OwcLgp9yAbVlrCyaRLTfYDItGw6KlBiyMgFMvQY1Uk4g0KQGs/CKaDERgHb2lTXYbgzhwEOZXIVPT3AsmEII4xKe5pfIZzBuRFxiQ7ihBn9vwkDw==",
    "forma": "請選擇遊戲",
    "Lotto649Control_history$txtNO": "", # 配入空字串
    "Lotto649Control_history$chk": "radYM",
    "Lotto649Control_history$dropYear": 108,
    "Lotto649Control_history$dropMonth": 1,
    "Lotto649Control_history$btnSubmit": "查詢"
}

res = requests.post(url,headers=headers,data=payload)  ### data 為要攜帶的參數
# print(res.text)
soup = bs(res.text,"lxml")
# print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>大樂透 各期獎號與開獎結果</title>
<link href="/css1.css" rel="stylesheet" type="text/css"/>
<script type="text/JavaScript">

<!--

     if (navigator.appName.indexOf("Internet Explorer") != -1)

         document.onmousedown = noSourceExplorer;

     function noSourceExplorer() {

         if (event.button == 2 | event.button == 3) {

             alert("版權所有 台灣彩券");

             //location.replace("http://www.taiwanlottery.com.tw/");

         }

     }

-->

</script>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.go